In [1]:
### Loading Credentials from local file; 
### this cell is meant to be deleted before publishing
import yaml

with open("../creds.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

uri = cfg["sonar_creds"]["uri"]
user = cfg["sonar_creds"]["user"]
password = cfg["sonar_creds"]["pass"]

# Check Physiology related topic terms

Search for "hysiolog" as substring for Physiology to retrieve every possible string containing.


In [2]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(uri, auth=(user, password))

query = """
MATCH (t:TopicTerm)
WHERE t.Name CONTAINS "hysiolog"
RETURN DISTINCT(t.Name)
"""

with driver.session() as session:
    all_physiology_terms = session.run(query).data()
    
all_physiology_terms

[{'(t.Name)': 'Arbeitsphysiologie'},
 {'(t.Name)': 'Neurophysiologie'},
 {'(t.Name)': 'Pathophysiologie'},
 {'(t.Name)': 'Pflanzenphysiologie'},
 {'(t.Name)': 'Physiologie'},
 {'(t.Name)': 'Sinnesphysiologie'},
 {'(t.Name)': 'Tierphysiologie'},
 {'(t.Name)': 'Physiologische Chemie'},
 {'(t.Name)': 'Physiologische Psychologie'},
 {'(t.Name)': 'Sprachphysiologie'},
 {'(t.Name)': 'Sportphysiologie'},
 {'(t.Name)': 'Leistungsphysiologie'},
 {'(t.Name)': 'Physiologische Psychiatrie'},
 {'(t.Name)': 'Elektrophysiologie'},
 {'(t.Name)': 'Altersphysiologie'},
 {'(t.Name)': 'Bewegungsphysiologie'},
 {'(t.Name)': 'Entwicklungsphysiologie'},
 {'(t.Name)': 'Ernährungsphysiologie'},
 {'(t.Name)': 'Ertragsphysiologie'},
 {'(t.Name)': 'Histophysiologie'},
 {'(t.Name)': 'Höhenphysiologie'},
 {'(t.Name)': 'Nacherntephysiologie'},
 {'(t.Name)': 'Physiologische Optik'},
 {'(t.Name)': 'Physiologische Uhr'},
 {'(t.Name)': 'Psychophysiologische Diagnostik'},
 {'(t.Name)': 'Stoffwechselphysiologie'},
 {'(t

With this in mind we can create a full network that contains every person connected to any kind of Physiological topic term. Also, we use the function `apoc.algo.cover(n)` to retrieve any kind of relationship between the persons connected to Physiological topic terms.

In [3]:
from helper_functions.helper_fun import to_nx_graph

query = """
MATCH (t:TopicTerm)-[r]-(n:PerName)
WHERE t.Name CONTAINS "hysiolog"
RETURN *
"""

driver = GraphDatabase.driver(uri, auth=(user, password))

G = to_nx_graph(neo4j_driver = driver, 
                query = query)

Check which topic terms aren't present in query result:

In [5]:
import numpy as np

relevant_topics = []
for node in list(G.nodes):
    if G.nodes[node]["type"] == "TopicTerm":
        relevant_topics.append((G.nodes[node]["label"]))

np.setdiff1d([d["(t.Name)"] for d in all_physiology_terms], relevant_topics)

array(['Altersphysiologie', 'Bewegungsphysiologie',
       'Elektrophysiologische Untersuchung', 'Ertragsphysiologie',
       'Experimentelle Physiologie', 'Histophysiologie',
       'Ignaz-L.-Lieben-Preis für Physik, Chemie und Physiologie',
       'Muskelphysiologie', 'Physiologische Optik', 'Physiologische Uhr',
       'Psychophysiologische Diagnostik', 'Reizphysiologie',
       'Sprachphysiologie', 'Tauchphysiologie', 'Umweltphysiologie',
       'Vergleichende Neurophysiologie', 'Vergleichende Physiologie',
       'Zellphysiologie'], dtype='<U57')

@todo add colors to network below by node type

In [6]:
from helper_functions.helper_fun import to_nx_graph
from pyvis.network import Network

nt = Network('750px', '100%', notebook=True, directed = True)
nt.from_nx(G)
nt.set_edge_smooth("dynamic")
#nt.show('./html_networks/physiological_net.html')

# Full network query

In [49]:
%%time

from helper_functions.helper_fun import to_nx_graph

query = """
MATCH (t:TopicTerm)--(n:PerName)
WHERE t.Name CONTAINS "hysiolog"
WITH DISTINCT [x in collect(t)+collect(n)|id(x)] as collectedIds 
MATCH (n)-[rel1:RelationToPerName]-(n2)
WHERE id(n) in collectedIds
RETURN n, n2, rel1
LIMIT 10000
"""


driver = GraphDatabase.driver(uri, auth=(user, password))

G = to_nx_graph(neo4j_driver = driver, 
                query = query)

CPU times: user 3.07 s, sys: 81.8 ms, total: 3.15 s
Wall time: 3.86 s


In [50]:
G.number_of_edges()

9619

## Betweenness Centrality

In [51]:
%%time
import networkx as nx
betweenness = nx.betweenness_centrality(G)

CPU times: user 2min 2s, sys: 7.5 ms, total: 2min 2s
Wall time: 2min 2s


In [52]:
from operator import itemgetter

betweenness_sorted = sorted(betweenness.items(), key = itemgetter(1), reverse = True)
top_betweenness = betweenness_sorted[:20]

In [54]:
for i in top_betweenness: 
    degree = betweenness[i[0]] # Use degree_dict to access a node's degree, see footnote 2
    print("Name:", G.nodes(data = True)[i[0]]["label"], "| Betweenness Centrality:", i[1])

Name: Eccles, John C. | Betweenness Centrality: 0.03965229437712333
Name: Weber, Ernst Heinrich | Betweenness Centrality: 0.019406124974935234
Name: Engelmann, Wilhelm | Betweenness Centrality: 0.018738346501259856
Name: Sömmerring, Samuel Thomas von | Betweenness Centrality: 0.017893257091632652
Name: None | Betweenness Centrality: 0.010001680253670908
Name: Frey, Max von | Betweenness Centrality: 0.009720930048847786
Name: Ludwig, Carl | Betweenness Centrality: 0.00878704841846009
Name: Zuntz, Nathan | Betweenness Centrality: 0.00774113642503953
Name: Hermann, Ludimar | Betweenness Centrality: 0.0076126218401466016
Name: Trendelenburg, Wilhelm | Betweenness Centrality: 0.005407824591234358
Name: Handbuch der Physiologie der Bewegungsapparate | Betweenness Centrality: 0.005234866959360121
Name: Schmidt, Robert F. | Betweenness Centrality: 0.004979847310878396
Name: Tiedemann, Friedrich | Betweenness Centrality: 0.00487144399698474
Name: Handbuch der Physiologie des Kreislaufs, der At

In [46]:
G.nodes(data = True)[2950836]["label"]

'Zuntz, Nathan'

## 7 Physiologists Network

* Gustav Fritsch (DE-588)115568808
* Eduard Hitzig (DE-588)116917423
* Hermann Munk (DE-588)117185930
* Nathan Zuntz (DE-588)118896202
* Friedrich Goltz (DE-588)116764694
* Adolf Fick (DE-588)118800000
* Jacques Loeb (DE-588)119133628

In [ ]:
physiologists = ["(DE-588)115568808", "(DE-588)116917423", "(DE-588)117185930", "(DE-588)118896202", 
                 "(DE-588)116764694", "(DE-588)118800000", "(DE-588)119133628"]

